In [15]:
# Required modules
import pygame, sys
from pygame.locals import *
import random, time

# Initialize pygame
pygame.init()

# Frames per second settings
FPS = 60
clock = pygame.time.Clock()

# Define colors
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

# Game variables
WIDTH = 400
HEIGHT = 600
velocity = 3
points = 0
collected_coins = 0

# Load fonts
font = pygame.font.SysFont("Verdana", 20)
small_font = pygame.font.SysFont("Verdana", 20)
game_over_text = font.render("Game Over", True, BLACK)

# Load background image
road_bg = pygame.image.load("street.png")

# Set up display window
screen = pygame.display.set_mode((WIDTH, HEIGHT))
screen.fill(WHITE)
pygame.display.set_caption("Racer")

class Obstacle(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("enemy.png")
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, WIDTH - 40), 0)

    def move(self):
        global points
        self.rect.move_ip(0, velocity)
        if self.rect.top > HEIGHT:
            points += 1
            self.rect.top = 0
            self.rect.center = (random.randint(40, WIDTH - 40), 0)

# Added a Coin class to manage coin spawning and tracking collected coins
speed_boost_1, speed_boost_2, speed_boost_3, speed_boost_4, speed_boost_5 = False, False, False, False, False
class Coin(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("coin.png")
        self.image = pygame.transform.scale(self.image, (40, 40))
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(40, WIDTH - 40), random.randint(40, HEIGHT - 40))

    def move(self):
        global collected_coins, velocity
        # Assign different coin values based on their position
        if self.rect.bottom < HEIGHT // 3:
            collected_coins += 3
        elif self.rect.bottom < HEIGHT // 1.5:
            collected_coins += 2
        else:
            collected_coins += 1

        # Adjust speed based on collected coins
        global speed_boost_1, speed_boost_2, speed_boost_3, speed_boost_4, speed_boost_5
        if not speed_boost_1 and collected_coins >= 10:
            velocity += 1
            speed_boost_1 = True
        if not speed_boost_2 and collected_coins >= 20:
            velocity += 1
            speed_boost_2 = True
        if not speed_boost_3 and collected_coins >= 30:
            velocity += 1
            speed_boost_3 = True
        if not speed_boost_4 and collected_coins >= 40:
            velocity += 1
            speed_boost_4 = True
        if not speed_boost_5 and collected_coins >= 50:
            velocity += 1
            speed_boost_5 = True

        # Reset coin position after collection
        self.rect.top = random.randint(40, WIDTH - 40)
        self.rect.center = (random.randint(40, WIDTH - 40), random.randint(40, HEIGHT - 40))

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Racer.png")
        self.rect = self.image.get_rect()
        self.rect.center = (160, 520)

    def move(self):
        keys = pygame.key.get_pressed()
        if self.rect.left > 0 and keys[K_LEFT]:
            self.rect.move_ip(-5, 0)
        if self.rect.right < WIDTH and keys[K_RIGHT]:
            self.rect.move_ip(5, 0)
        if self.rect.top > 0 and keys[K_UP]:
            self.rect.move_ip(0, -5)
        if self.rect.bottom < HEIGHT and keys[K_DOWN]:
            self.rect.move_ip(0, 5)

# Initialize game objects
player = Player()
enemy = Obstacle()
coin = Coin()

# Group enemies, coins, and all sprites
obstacles = pygame.sprite.Group()
obstacles.add(enemy)
coins_group = pygame.sprite.Group()
coins_group.add(coin)
all_sprites = pygame.sprite.Group()
all_sprites.add(player, enemy, coin)

# Custom event to gradually increase speed
INCREASE_SPEED = pygame.USEREVENT + 1
pygame.time.set_timer(INCREASE_SPEED, 1000)

def game_over_screen():
    screen.fill(RED)
    screen.blit(game_over_text, (30, 250))
    pygame.display.update()

    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == KEYDOWN:
                if event.key == K_SPACE:  # Restart game when spacebar is pressed
                    return True
                elif event.key == K_ESCAPE:  # Quit game when ESC is pressed
                    return False

def handle_crash():
    time.sleep(2)

# Initialize background scrolling
bg_y = 0  

while True:
    for event in pygame.event.get():
        if event.type == INCREASE_SPEED:
            velocity += 0.1
        if event.type == QUIT:
            pygame.quit()
            sys.exit()

    # Check collision between player and enemy
    if pygame.sprite.spritecollideany(player, obstacles):
        continue_game = handle_crash()
        if not continue_game:
            pygame.quit()
            sys.exit()

    # Scroll the background to create motion effect
    bg_y = (bg_y + velocity) % road_bg.get_height()
    screen.blit(road_bg, (0, bg_y))
    screen.blit(road_bg, (0, bg_y - road_bg.get_height()))

    # Display the score
    score_display = small_font.render(str(points), True, BLACK)
    screen.blit(score_display, (10, 10))

    # Display collected coins
    coin_display = small_font.render(str(collected_coins), True, BLACK)
    screen.blit(coin_display, (370, 10))

    # Update and redraw all game objects
    for entity in all_sprites:
        screen.blit(entity.image, entity.rect)

        # Check if player collects a coin
        if entity == coin and pygame.sprite.spritecollideany(player, coins_group):
            entity.move()
        else:
            entity.move()

    # Move all enemies
    for obstacle in obstacles:
        obstacle.move()

    # Move and respawn coins when they leave the screen
    for coin in coins_group:
        coin.rect.y += velocity
        if coin.rect.top > HEIGHT:
            coin.rect.y = -coin.rect.height
            coin.rect.x = random.randint(40, WIDTH - 40)

    pygame.display.update()
    clock.tick(FPS)


SystemExit: 